In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg16_bn", pretrained=True)
# model.classifier[4] = nn.Sigmoid()
model.load_state_dict(torch.load("VGG16_cifar100_sigmoid_origin_1.pth"))
hout = np.load('Hout_VGG16_train.npy', allow_pickle=True)
hout = [np.array(h, dtype=np.float32) for h in hout]

Using cache found in C:\Users\huawei/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


In [3]:
hout0 = [np.round(h, 0) for h in hout]
# 0-1 0.59776796
# 0-2 -0.02472771
# 0-4 -0.43763658
# 0-8 -0.59199387
# 0-16 -0.5738108
# 0-32 -0.5171188
# WHY?

In [4]:
def p(x):
    unique, counts = np.unique(x, axis=0, return_counts=True)
    return unique, counts / len(x)

def H(p):
    return -np.sum(p * np.log2(p))

def MI(x, y, xy):
    return H(x) + H(p(y)[1]) - H(p(xy)[1])

# x = np.array([i for i in range(len(hout)) for _ in range(len(hout01[i]))])
x = np.array([len(h) for h in hout0])
x = x / x.sum()
y = np.concatenate(hout0, axis=0)
xy = np.concatenate([np.c_[hout0[i],[i for _ in hout0[i]]] for i in range(len(hout0))], axis=0)

# H(x), H(p(y)[1]), H(p(xy)[1]), MI(x, y, xy)

In [5]:
def MIxy(slice):
    return MI(x, y[:,slice], xy[:,slice+[-1]])

# 不同神经元与最终输出的互信息
MIs = [MIxy([i]) for i in range(y.shape[1])]
MIs = np.array(MIs)

In [6]:
y = np.concatenate(hout, axis=0)
hvar = [y[:, i].var() for i in range(y.shape[1])] # Task Variance
hvar = np.array(hvar)

In [7]:
b = model.classifier[6].bias.data.cpu().numpy()
W = model.classifier[6].weight.data.cpu().numpy()
B = np.tile(b/W.shape[1], (W.shape[1],1)).T # 认为每条连接对bias的贡献相同
b_ = B / W
b_ = b_.sum(axis=0)

In [9]:
np.corrcoef([MIs, hvar, b_])

array([[1.        , 0.59776796, 0.08179464],
       [0.59776796, 1.        , 0.09920422],
       [0.08179464, 0.09920422, 1.        ]])

In [10]:
from scipy import stats
# 斯皮尔曼相关系数，皮尔逊相关系数
stats.spearmanr(MIs, hvar), stats.pearsonr(MIs, hvar), \
stats.spearmanr(MIs, b_), stats.pearsonr(MIs, b_), \
stats.spearmanr(hvar, b_), stats.pearsonr(hvar, b_)

(SignificanceResult(statistic=0.5830170102768336, pvalue=6.020445122643349e-48),
 PearsonRResult(statistic=0.5977679634481455, pvalue=6.418602819133077e-51),
 SignificanceResult(statistic=0.014682289342076653, pvalue=0.7403211071971914),
 PearsonRResult(statistic=0.08179463906241465, pvalue=0.06440331053605411),
 SignificanceResult(statistic=0.04720983036930225, pvalue=0.28632413217271147),
 PearsonRResult(statistic=0.09920421798866629, pvalue=0.024781708081683142))